In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time

import pandas as pd
from deltaNets import combustionML
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from dataScaling import LogScaler, AtanScaler, NoScaler
from boost_test import test_data

/home/eg/anaconda3/envs/my_dev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


precision: float32
Running Cantera version: 2.3.0
Running Cantera version: 2.3.0
Running Cantera version: 2.3.0


In [2]:
df_x, df_y = pickle.load(open('data/x_y_org.p', 'rb'))

In [47]:
df_x_org, df_y_org = pickle.load(open('data/x_y_org.p', 'rb'))
columns = df_x_org.columns
train_features = columns.drop(['f','N2','Rho','cp','Hs','dt'])

# initial conditions
n_H2 = sorted(list(map(float, set(df_x_org['f']))))
n_H2 = np.asarray(n_H2).reshape(-1, 1)

df_x = df_x_org[train_features]
df_y = df_y_org[train_features]

pca= PCA(n_components=3) # pca for 3D visualization
df_x_s=df_x.sample(frac=1) # take 10% data
# idx = (df_y/df_x <1.2).all(1)
# df_x_s = df_x.loc[idx]

# scaler = NoScaler()
scaler = StandardScaler()
train=pca.fit_transform(scaler.fit_transform(df_x_s))
# train = np.hstack((train,0*np.ones((train.shape[0],1)))) # add a label 
# train = np.hstack((train,),np.round((df_y/df_x).mean(1),1).reshape(-1,1))

In [48]:
np.round((df_y/df_x).mean(1),1)

0          5.6
1          4.7
2          1.6
3          1.3
4          1.2
5          1.2
6          1.1
7          1.1
8          1.1
9          1.1
10         1.1
11         1.1
12         1.1
13         1.1
14         1.1
15         1.1
16         1.1
17         1.1
18         1.1
19         1.1
20         1.1
21         1.1
22         1.1
23         1.1
24         1.1
25         1.0
26         1.0
27         1.0
28         1.0
29         1.0
          ... 
2352881    1.0
2352904    1.0
2352914    1.0
2352929    1.0
2352937    1.0
2352941    1.0
2352955    1.0
2352960    1.0
2352963    1.0
2352984    1.0
2352989    1.0
2353001    1.0
2353003    1.0
2353037    1.0
2353040    1.0
2353048    1.0
2353112    1.0
2353151    1.0
2353160    1.0
2353177    1.0
2353184    1.0
2353207    1.0
2353217    1.0
2353249    1.0
2353255    1.0
2353258    1.0
2353266    1.0
2353285    1.0
2353313    1.0
2353321    1.0
Length: 328408, dtype: float64

In [49]:
(df_y/df_x).head()

,H2,H,O2,OH,O,H2O,HO2,H2O2,T
0,1.0,0.782791,1.0,4.769054,4.834450,31.423876,0.785184,4.864314,1.0
1,1.0,1.807724,1.0,3.054794,3.394656,25.816300,1.813114,3.577869,1.0
2,1.0,1.420866,1.0,1.927018,1.992736,2.505715,1.424516,2.022977,1.0
3,1.0,1.246821,1.0,1.484600,1.495203,1.558799,1.249066,1.499840,1.0
4,1.0,1.227438,1.0,1.438707,1.440982,1.453921,1.229764,1.441956,1.0


In [50]:
df_pca=pd.DataFrame(train,columns=['x','y','z'])
df_pca['label']=np.round((df_y['O']/df_x['O']),1).reset_index()['O']

In [51]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_x)
df_pca['label']=kmeans.labels_

In [52]:
import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.tools as tls

# df_pca=df_pca.sample(frac=0.1)
df_pca_react=df_pca[df_pca['label']<5]
df_pca_s=df_pca_react.sample(10_000)

fig_db = {
    'data': [
        {'x': df_pca_s[df_pca_s['label']==label]['x'],
         'y': df_pca_s[df_pca_s['label']==label]['y'],
         'z': df_pca_s[df_pca_s['label']==label]['z'],
         'type':'scatter3d', 'mode': 'markers', 'marker':{'size':2}, 'name': label} for label in set(df_pca_react['label'])
        
    ],
    'layout': {
    }
}
pyoff.iplot(fig_db, filename='multiple-scatter')